### Librarias

In [73]:
import sympy as sp
import numpy as np
from numpy.linalg import inv
x = sp.Symbol('x')

### Funções auxiliares

In [74]:
def matrizdosCoeficientes( objeto, grauDoNumerador, grauDoDenominador):

    if type( objeto) == list or type( objeto) == tuple:
        if grauDoNumerador + grauDoDenominador + 1 <= len(objeto):
            A = sp.Matrix( np.zeros( ( grauDoDenominador, grauDoDenominador ) ) ) 
            
            for linha in range( 0, grauDoDenominador ):  
                
                for coluna in range(0, grauDoDenominador ):
                    if ( grauDoNumerador - grauDoDenominador + coluna + 1 + linha < 0 ):
                        A[ linha, coluna ] = 0
                
                    elif ( grauDoNumerador - grauDoDenominador + coluna + 1 + linha == 0 ):
                        A[ linha, coluna ] = objeto[0]
                
                    else:
                        an = objeto[ grauDoNumerador - grauDoDenominador + coluna + 1 + linha ]
                        A[ linha, coluna ] =  an
            return(A)
        
        else: return('O número de coeficientes não é suficiente.')
            
    else:
        
        A = sp.Matrix( np.zeros( ( grauDoDenominador, grauDoDenominador) ) ) 
        for linha in range( 0, grauDoDenominador ):  
            for coluna in range( 0, grauDoDenominador ):
                if ( grauDoNumerador - grauDoDenominador + coluna + linha + 1 < 0 ):
                    A[ linha, coluna ] = 0
                elif ( grauDoNumerador - grauDoDenominador + coluna + 1 + linha == 0 ):
                    derivada = sp.diff( objeto, x, 0 )
                    derivadaEmZero = derivada.subs( x, 0 )
                    A[ linha, coluna ] = derivadaEmZero / sp.Rational(sp.factorial( 0 ) )
                else:
                    derivada = sp.diff( objeto, x, grauDoNumerador - grauDoDenominador + coluna + linha + 1  )
                    derivadaEmZero =  derivada.subs( x, 0 )
                    A[ linha, coluna ] = derivadaEmZero /\
                                    sp.Rational(sp.factorial( grauDoNumerador - grauDoDenominador + coluna + linha + 1 ) )
       
        return ( A )

In [76]:
def matrizDosTermosIndependentes( objeto, grauDoNumerador, grauDoDenominador ):
 
    if type( objeto) == list or type( objeto) == tuple:
        if grauDoNumerador + grauDoDenominador + 1 <= len(objeto):
            # vetor nulo 1xq
            c = sp.Matrix( np.zeros(( 1, grauDoDenominador )) ) 
            # cálculo dos coeficientes da série de Taylor em x = 0 
            for coluna in range( 0, grauDoDenominador ):
                an = objeto[ grauDoNumerador + coluna +1]   
                c[ 0, coluna ] = ( -1 ) * an
                
            return (c)
    
        else:
            return('O número de coeficientes não é suficiente.')

    else:
        c = sp.Matrix( np.zeros( ( 1 , grauDoDenominador ) ) )   
        for coluna in range( 0, grauDoDenominador ):
            derivada = sp.diff( objeto, x, grauDoNumerador + coluna + 1 )
            derivadaEmZero = derivada.subs( x, 0 )
            c[ 0, coluna ] = (-1) *( derivadaEmZero / sp.Rational(sp.factorial( grauDoNumerador + coluna + 1 ) ) )
        return ( c )

In [77]:
def coeficientesDaFuncao( objeto, grauDoNumerador, grauDoDenominador ):

    if type( objeto) == list or type( objeto) == tuple:
        if grauDoNumerador + grauDoDenominador + 1 <= len(objeto):
            cA = sp.Matrix(np.zeros(( 1, grauDoNumerador + 1 )))
            for coluna in range( 0, grauDoNumerador + 1 ):
                if ( grauDoNumerador - coluna < 0 ):
                    cA[ coluna ] = 0
                else:
                    an = objeto[ grauDoNumerador - coluna ] 
                    cA[ coluna ] = an
                    
            return( cA )  
        else:
            return('O número de coeficientes não é suficiente.')
            
    else: 
        
        # cálculo dos coeficientes da série de Taylor de f em x = 0 
        cN = sp.Matrix( np.zeros( ( 1, grauDoNumerador + 1 ) ) )
        for coluna in range( 0, grauDoNumerador + 1 ):
            if ( grauDoNumerador - coluna < 0 ):
                cN[ coluna ] = 0
            else:
                fn = sp.diff( objeto, x, grauDoNumerador - coluna )
                cN[ coluna ] = fn.subs( x, 0 )/sp.Rational( sp.factorial( grauDoNumerador - coluna ) )
        return ( cN )

### Cálculo do Padé  \[ p / q \]

In [81]:
def padeDiretoPrecisaoInfinita( objeto, grauDoNumerador, grauDoDenominador ):
    x = sp.Symbol('x')
    
    # Matriz dos Coeficientes de Ab = c
    A = matrizdosCoeficientes( objeto, grauDoNumerador, grauDoDenominador )

    
    if(A is None): return
    
    if (type(A) is str) : return(A)
    
    if(A == 0): return("A matriz dos coeficientes não é invertível ")
        
    # Se a matriz A é invertível    
    if ( A.det() != 0 ):
        A1 = A.inv("LU")
        Bn = matrizDosTermosIndependentes( objeto, grauDoNumerador, grauDoDenominador ) * A.inv("LU")
        Dx = sp.Matrix( np.zeros( ( grauDoDenominador + 1, 1 ) ) )
        
        for linha in range( 0, grauDoDenominador + 1 ):
                Dx[ linha ] = x**(linha)
                
        bn = sp.Matrix( np.zeros( ( 1, grauDoDenominador + 1 ) ) )
        bn[0] = 1
        
        for coluna in range( 1, grauDoDenominador + 1 ):
            bn[ coluna ] = Bn[ -coluna ]

        Denominador = sp.Function( 'Denominador' )
        Denominador = bn * Dx
        

       
        cn = sp.Matrix( np.zeros( ( 1, grauDoNumerador + 1 ) ) )
        Nx = sp.Matrix( np.zeros( ( grauDoNumerador + 1 ) ) )
        for coluna in range( 0, grauDoNumerador + 1 ):
                Nx[ coluna ] = x**( coluna )
         
        # Se o grauDoNumerador é menor do que o grauDoDenominador
        if ( grauDoNumerador < grauDoDenominador ):
            x = sp.Symbol('x')
            coluna = 1
          
            while( coluna <= grauDoNumerador + 1 ):
                
                An = sp.Matrix( coeficientesDaFuncao( objeto, grauDoNumerador, grauDoDenominador )[ -coluna: ] )
                Bn = sp.Matrix( bn[ 0:coluna ] )
                cn[ coluna-1 ] = sp.Transpose( An ) * Bn
                
                coluna += 1
                
                
            Numerador = sp.Function( 'Numerador' )
            Numerador = cn * Nx
            
            R = Numerador/Denominador
            Pade = sp.Function( 'Pade')
            Pade = R[0]
            
        # Se o grauDoNumerador é maior do que o grauDoDenominador
        else:
            x = sp.Symbol('x')
            coluna = 1
      
            while( coluna <= grauDoDenominador + 1  ):
                
                    An = sp.Matrix( coeficientesDaFuncao( objeto, grauDoNumerador, grauDoDenominador )[ -coluna: ] )
                    Bn = sp.Matrix( bn[ 0:coluna ] )
                    cn[ coluna-1 ]=sp.Transpose( An ) * Bn
                    
                    coluna += 1
                   
                    
            j = -1
          
            while ( coluna <= grauDoNumerador + 1):
                
                    An = sp.Matrix( coeficientesDaFuncao( objeto, grauDoNumerador, grauDoDenominador )[ -coluna:j ] )
                    Bn = sp.Matrix( bn[ 0: ] )
                    cn[ coluna - 1 ] = sp.Transpose( An ) * Bn
                    
                    coluna += 1
                    j -= 1
          
                    
            Numerador = sp.Function( 'Numerador' )
            Numerador = cn * Nx
            R = sp.Function( 'R' )
            R = Numerador/Denominador
            
            Pade = sp.Function( 'Pade')
            Pade = R[0]
            
        return ( Pade )
    
    # Se a matriz não é invertível
    else: return('A matrix não é invertível' )